In [22]:
import os
import pandas as pd

In [23]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [24]:
folder("pcaps")

In [25]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [26]:
files_add=find_the_way('./','.pcap')
files_add=[ './pcaps\\http-flood.pcap']

In [27]:
for i in files_add:
    temp=i[8:]
    temp=temp.replace(" ","_")
    temp=temp.replace("\\","-")
    temp=f"./pcaps/{temp}"
    #os.rename(i,temp)
    print(temp)

./pcaps/http-flood.pcap


In [28]:
files_add=find_the_way('./','.pcap')
uzun=len(files_add)
uzun

4

In [29]:
df = pd.read_csv( "dataset_description.csv")
df

,No,File Name,Category,Sub-category,Rule
0,1,udp_flood.pcap,Flood,UDP,(ip.proto == 17) && (udp.dstport == 80)
1,2,SYN.pcap,Flood,SYN,ip.dst==10.128.0.2 && ip.src==10.0.0.2
2,3,ACK_v3.pcap,Flood,ACK,tcp.flags.ack == 1
3,4,http-flood.pcap,Flood,HTTP,ip.src==10.0.0.2 && ip.dst==10.128.0.2


In [30]:
df.columns

Index(['No', 'File Name', 'Category', 'Sub-category', 'Rule'], dtype='object')

In [31]:
name=df['File Name']
rule=df['Rule']
cat=df['Category']
subcat=df['Sub-category']

In [32]:
name

0     udp_flood.pcap
1           SYN.pcap
2        ACK_v3.pcap
3    http-flood.pcap
Name: File Name, dtype: object

In [33]:
for i in range(len(name)):
    add=str(cat[i])+"@"+str(subcat[i])+"_"
    add=add.replace(" ","_")
    command='tshark -Y \"'+str(rule[i])+"\" -r ./pcaps/"+str(name[i])+" -T fields -e frame.number > ./pcaps/attack_"+str(name[i])[:-4]+"csv"
    os.system(command)
    command='tshark -Y \"!('+str(rule[i])+")\" -r ./pcaps/"+str(name[i])+" -T fields -e frame.number > ./pcaps/normal_"+str(name[i])[:-4]+"csv"
    os.system(command)
    files_add=find_the_way('./','.pcap')
    if uzun==len(files_add):    
        print(command,"\n")
    uzun=len(files_add) 
 
    

tshark -Y "!( (ip.proto == 17) && (udp.dstport == 80))" -r ./pcaps/udp_flood.pcap -T fields -e frame.number > ./pcaps/normal_udp_flood.csv 

tshark -Y "!(ip.dst==10.128.0.2 && ip.src==10.0.0.2)" -r ./pcaps/SYN.pcap -T fields -e frame.number > ./pcaps/normal_SYN.csv 

tshark -Y "!(tcp.flags.ack == 1)" -r ./pcaps/ACK_v3.pcap -T fields -e frame.number > ./pcaps/normal_ACK_v3.csv 

tshark -Y "!(ip.src==10.0.0.2 && ip.dst==10.128.0.2)" -r ./pcaps/http-flood.pcap -T fields -e frame.number > ./pcaps/normal_http-flood.csv 



# MERGE CSVS

In [34]:
name_list=find_the_way('./pcaps','.csv')

for i in name_list:
    boyut=os.path.getsize(i)
    if boyut==0:
        os.remove(i)
        
    

In [35]:
label_files=[]
for i in name_list:
    temp=i[15:]
    if temp not in label_files:
        label_files.append(temp)
    

In [36]:
for i in label_files:

    name=find_the_way('./pcaps',i)
    print(name)

    dfA = pd.read_csv(name[0], header=None)
    dfA=dfA.rename(columns={0: "PacketNumber"})
    dfA["Label"]=[1] * len(dfA)
    
    try:
        dfN = pd.read_csv(name[1], header=None)
        dfN=dfN.rename(columns={0: "PacketNumber"})    
        dfN["Label"]=[0] * len(dfN) 
        df = pd.concat([dfA,dfN])
        df=df.sort_values('PacketNumber')
        df.to_csv(f"./pcaps/{i}",  index=False)
        print(i,df.groupby("Label").size(),"\n\n\n")
        os.remove(f"./pcaps\\normal_{i}")
        os.remove(f"./pcaps\\attack_{i}")
    except:
        dfA.to_csv(f"./pcaps/{i}",  index=False)
        print(i,dfA.groupby("Label").size(),"\n\n\n")
        

        


['./pcaps\\attack_ACK_v3.csv', './pcaps\\normal_ACK_v3.csv']
ACK_v3.csv Label
0    14754
1    14765
dtype: int64 



['./pcaps\\attack_http-flood.csv', './pcaps\\normal_http-flood.csv']
http-flood.csv Label
0    1002
1    1429
dtype: int64 



['./pcaps\\attack_SYN.csv', './pcaps\\normal_SYN.csv']
SYN.csv Label
0    3317
1    1658
dtype: int64 



['./pcaps\\attack_udp_flood.csv']
udp_flood.csv Label
1    999
dtype: int64 





In [37]:
#!shutdown /s /t 36

In [106]:
df

,PacketNumber,Label
0,1,1
0,2,0
1,3,0
1,4,1
2,5,0
...,...,...
1657,4971,1
3313,4972,0
3314,4973,0
3315,4974,0
